IMPORT LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
import pandas as pd
from scipy import stats
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

LOAD DATA SET

In [3]:
df=pd.read_csv('/content/store.csv')

In [6]:
df.columns  # to check column name

Index(['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

UPLOAD TRAIN DATASET

In [7]:
train=pd.read_csv('/content/train.csv')
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [8]:
train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')

In [9]:
test=pd.read_csv('/content/test.csv')

In [10]:
train.isnull().sum() # to check null value in train columns

,0
Store,0
DayOfWeek,0
Date,0
Sales,0
Customers,0
Open,0
Promo,0
StateHoliday,0
SchoolHoliday,0


In [11]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


Key Observation-

There is a large variance in sales and customers, indicating different store sizes or performance levels.

Stores are usually open, with some closed days affecting sales.

Promotions and school holidays impact sales, but promotions are more frequent than school holidays.

The dataset is right-skewed with a large max value (41,551).


In [12]:
train.info() # Information about the train dataset


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [13]:
# find out the categorical columns
categorical_columns=train.select_dtypes(include='object')
categorical_columns.head(4)

,Date,StateHoliday
0,2015-07-31,0
1,2015-07-31,0
2,2015-07-31,0
3,2015-07-31,0


In [14]:
#convert date into date time
#train['Date'] = pd.to_datetime(train['Date'])
train['Date'] = pd.to_datetime(train['Date']).apply(lambda date: date.toordinal())


In [15]:
# convert StateHoliday object to int
train['StateHoliday'] = train['StateHoliday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})

In [16]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Store          1017209 non-null  int64  
 1   DayOfWeek      1017209 non-null  int64  
 2   Date           1017209 non-null  int64  
 3   Sales          1017209 non-null  int64  
 4   Customers      1017209 non-null  int64  
 5   Open           1017209 non-null  int64  
 6   Promo          1017209 non-null  int64  
 7   StateHoliday   886137 non-null   float64
 8   SchoolHoliday  1017209 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 69.8 MB


In [18]:
train['StateHoliday'].unique()

array([0., 1., 2., 3.])

In [17]:
# handle null value of Stateholiday columns
train['StateHoliday'].fillna(0, inplace=True)

In [19]:
# correlations
corr_matrix = train.corr()
corr_matrix

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Store,1.000000,-0.000008,0.000842,0.005126,0.024325,-0.000047,0.000058,0.000362,0.000641
DayOfWeek,-0.000008,1.000000,0.000183,-0.462125,-0.386445,-0.528963,-0.392925,-0.044265,-0.205388
Date,0.000842,0.000183,1.000000,0.041904,0.012679,-0.000157,0.017249,0.008197,0.001068
Sales,0.005126,-0.462125,0.041904,1.000000,0.894711,0.678472,0.452345,-0.229029,0.085124
Customers,0.024325,-0.386445,0.012679,0.894711,1.000000,0.616768,0.316169,-0.204295,0.071568
Open,-0.000047,-0.528963,-0.000157,0.678472,0.616768,1.000000,0.295042,-0.340893,0.086171
Promo,0.000058,-0.392925,0.017249,0.452345,0.316169,0.295042,1.000000,-0.024157,0.067483
StateHoliday,0.000362,-0.044265,0.008197,-0.229029,-0.204295,-0.340893,-0.024157,1.000000,0.196424
SchoolHoliday,0.000641,-0.205388,0.001068,0.085124,0.071568,0.086171,0.067483,0.196424,1.000000


This is a correlation matrix, showing the relationships between different numerical variables in dataset.

Sales & Customers (+0.89): Sales are highly dependent on the number of customers.

Open & Sales (+0.68): Open stores generate more sales (which makes sense).

Promo & Sales (+0.45): Promotions have a positive impact on sales.

Sales are mostly driven by the number of customers and promotions.

Weekdays impact sales significantly, with lower sales on certain days.

Holidays affect sales, but school holidays have a much smaller effect than state holidays.



In [4]:
# model fitting
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [20]:
x=train.drop('Sales',axis=1)
y=train['Sales']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((813767, 8), (203442, 8), (813767,), (203442,))

In [23]:
model=RandomForestRegressor()
model.fit(X_train,y_train)

RandomForestRegressor()

In [24]:
#prediction
y_pred=model.predict(X_test)

In [25]:
#accuracy
accuracy=r2_score(y_test,y_pred)
accuracy

0.9590956901135751

In [27]:
#another model like xgboost
from xgboost import XGBRegressor  # Import XGBRegressor from xgboost
model_XGB=XGBRegressor()
model_XGB.fit(X_train,y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [28]:
#Prediction
y_pred=model_XGB.predict(X_test)


In [30]:
#accuracy
accuracy=r2_score(y_test,y_pred)
accuracy


0.9326215982437134

In [31]:
# Try one more model
from sklearn.linear_model import LinearRegression
model_LR=LinearRegression()
model_LR.fit(X_train,y_train)

LinearRegression()

In [32]:
#prediction
y_pred=model_LR.predict(X_test)

In [33]:
# accuracy
accuracy=r2_score(y_test,y_pred)
accuracy

0.8522067682569826

In [ ]:
# I have tried 3 model buy getting hightest accuracy in Random forest regressor (RandomForestRegressor)

In [ ]:
#mse
import numpy as np
mse=mean_squared_error(y_test,y_pred)
mse

603661.4326557171

In [ ]:
#mae
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test,y_pred)
mae

467.29384925433277

In [ ]:
#mae
from sklearn.metrics import mean_absolute_error
mae=mean_absolute_error(y_test,y_pred)
mae

467.29384925433277

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, Huber

# # Create an instance of the Huber class with a delta value
# huber_loss_fn = Huber(delta=1.0)
# huber_loss = huber_loss_fn(y_test, y_pred).numpy()

# # Print loss values

# print(f"Huber Loss: {huber_loss}")

Huber Loss: 466.87921142578125


In [ ]:
# train_mse = mean_squared_error(y_train, model.predict(X_train))
# test_mse = mean_squared_error(y_test, model.predict(X_test))

# print(f"Training MSE: {train_mse}")
# print(f"Test MSE: {test_mse}")

Training MSE: 84271.89545154336
Test MSE: 603661.4326557171


In [ ]:
# train_r2 = r2_score(y_train, model.predict(X_train))
# test_r2 = r2_score(y_test, model.predict(X_test))

# print(f"Training R² Score: {train_r2}")
# print(f"Test R² Score: {test_r2}")

Training R² Score: 0.9943175457934416
Test R² Score: 0.9591811648738408


In [ ]:
# # Check overfitting or underfitting
# if train_r2 > 0.9 and test_r2 < 0.7:
#     print("Model is overfitting.")
# elif train_r2 < 0.7 and test_r2 < 0.7:
#     print("Model is underfitting.")
# else:
#     print("Model is well-fitted.")

Model is well-fitted.


In [ ]:
#dump the model
import pickle
pickle.dump(model,open('retailstore.pkl','wb'))

In [ ]:
# 2.4 post prediction analysis

In [ ]:
# Assuming you used a Random Forest or XGBoost model
 feature_importances = model.feature_importances_

In [ ]:

# # Create DataFrame for visualization
import pandas as pd
feature_names = X_train.columns
feat_imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feat_imp_df = feat_imp_df.sort_values(by="Importance", ascending=False)


In [ ]:
# # Plot Feature Importance
plt.figure(figsize=(10, 5))
sns.barplot(x='Importance', y='Feature', data=feat_imp_df, palette="viridis")
plt.title("Feature Importance")
plt.show()